In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/Tugas NLP/Language Modeling/rhyme.txt'

Mounted at /content/drive


In [ ]:
import string
import re
from os import listdir
import numpy as np
from numpy import array
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
import pandas as pd
df = pd.read_csv(filename)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
data = df.to_dict(orient='records')
values = [value['text'] for value in data if value['airline_sentiment_confidence'] >= 0.5]
values[:10]

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it",
 "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
 '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
 '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
 "@virginamerica Well, I didn't…but NOW I DO! :-D",
 "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",
 '@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24']

In [ ]:
def clean_text(text):
  tokens = text.split()
  # regular expression to match any punctuation character (excluding apostrophes)
  re_punc = re.compile('[%s]' % re.escape(string.punctuation.replace("'", "")))
  # regular expression to match camel case words
  camel_case_regex = re.compile(r'([a-z])([A-Z])')

  # remove string that contains @ from each word
  tokens = [w for w in tokens if "@" not in w]
  # remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
  tokens = [camel_case_regex.sub('', w) for w in tokens]
  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  # filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens


def add_text_to_vocab(text, vocab):
  tokens = clean_text(text)
  vocab.update(tokens)

def load_text(values, vocab):
  for text in values:
    add_text_to_vocab(text, vocab)

In [ ]:
vocab = Counter()
load_text(values, vocab)

In [ ]:
vocab.most_common(50)

[('flight', 3111),
 ('get', 1277),
 ('Cancelled', 1054),
 ('service', 908),
 ('time', 737),
 ('Flight', 722),
 ('help', 709),
 ('customer', 685),
 ('hours', 646),
 ('amp', 630),
 ('hold', 628),
 ('flights', 601),
 ('plane', 587),
 ('thanks', 521),
 ('one', 507),
 ('us', 507),
 ('Thanks', 503),
 ('Flightled', 500),
 ('back', 499),
 ('would', 492),
 ('delayed', 486),
 ('call', 480),
 ('still', 473),
 ('gate', 464),
 ('need', 462),
 ('bag', 460),
 ('hour', 450),
 ('Late', 425),
 ('got', 413),
 ('like', 406),
 ('phone', 403),
 ('today', 380),
 ('airline', 377),
 ('guys', 357),
 ('fly', 356),
 ('please', 346),
 ('know', 343),
 ('You', 343),
 ('waiting', 339),
 ('No', 319),
 ('way', 319),
 ('day', 308),
 ('wait', 306),
 ('change', 303),
 ('Thank', 302),
 ('trying', 300),
 ('airport', 297),
 ('going', 296),
 ('make', 290),
 ('minutes', 288)]

In [ ]:
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(tokens)

['What', 'said', 'today', 'Must', 'mean', 'need', 'take', 'another', 'trip', 'really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'faces', 'amp', 'little', 'recourse', 'big', 'bad', 'thing', 'seriously', 'would', 'pay', 'flight', 'seats', 'playing', 'flying', 'VA', 'yes', 'nearly', 'every', 'time', 'fly', 'VX', 'go', 'away', 'Really', 'missed', 'prime', 'opportunity', 'Men', 'Without', 'Hats', 'Well', 'NOW', 'DO', 'amazing', 'arrived', 'hour', 'early', 'good', 'know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teens', 'pretty', 'much', 'better', 'This', 'great', 'deal', 'Already', 'thinking', 'even', 'gone', 'yet', 'fabulous', 'skies', 'stress', 'travel', 'Thanks', 'schedule', 'still', 'MIA', 'So', 'excited', 'first', 'cross', 'country', 'LAX', 'MCO', 'heard', 'nothing', 'things', 'Virgin', 'America', 'flew', 'NYC', 'SFO', 'last', 'week', 'fully', 'sit', 'seat', 'due', 'two', 'large', 'gentleman', 'either', 'side', 'HELP', 'awesome', 'PLEASE', 'want', 'fares'

In [ ]:
def save_list(lines, filename):
  # convert lines to a single blob of text
  data = '\n'.join(lines)
  # open file
  file = open(filename, 'w')
  # write text
  file.write(data)
  # close file
  file.close()

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

In [ ]:
vocab_filename = 'vocab.txt'
# save tokens to a vocabulary file
save_list(tokens, vocab_filename)

In [ ]:
# load the vocabulary
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
len(vocab)

6560

# Train CNN with embedding layer

In [ ]:
def clean_tweet(text, vocab):
  tweet = clean_text(text)
  tweet = [w for w in tweet if w in vocab]
  return (' ').join(tweet)

def process_docs_tweet(data, category, vocab):
  tweet_data = [tweet['text'] for tweet in data if tweet['airline_sentiment'] == category]
  tweet_data = [clean_tweet(tweet, vocab) for tweet in tweet_data]
  return tweet_data

def load_clean_tweet(data, vocab):
  categories = ["negative", "neutral", "positive"]
  neg_tweet = process_docs_tweet(data, categories[0], vocab)
  neu_tweet = process_docs_tweet(data, categories[1], vocab)
  pos_tweet = process_docs_tweet(data, categories[2], vocab)
  docs = neg_tweet + neu_tweet + pos_tweet
  labels = array([0 for _ in range(len(neg_tweet))] + [1 for _ in range(len(neu_tweet))] + [2 for _ in range(len(pos_tweet))])
  return docs, labels

# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
  # integer encode
  encoded = tokenizer.texts_to_sequences(docs)
  # pad sequences
  padded = pad_sequences(encoded, maxlen=max_length, padding='post')
  return padded

# define the model
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize defined model
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

In [ ]:
data = df.to_dict(orient='records')
data = [value for value in data if value['airline_sentiment_confidence'] >= 0.5]
train_docs, ytrain = load_clean_tweet(data, vocab)
# Convert ytrain to one-hot encoding
ytrain = to_categorical(ytrain, num_classes=3)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)
# fit network
model.fit(Xtrain, ytrain, epochs=30, verbose=2)
# save the model
model.save('model.h5')

Vocabulary size: 5346
Maximum length: 22
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 22, 100)           534600    
                                                                 
 conv1d (Conv1D)             (None, 15, 32)            25632     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 32)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 10)                2250      
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
               

In [ ]:
def predict_sentiment(review, vocab, tokenizer, max_length, model):
  # clean review
  line = clean_tweet(review, vocab)
  # encode and pad review
  padded = encode_docs(tokenizer, max_length, [line])
  # predict sentiment
  yhat = model.predict(padded, verbose=0)
  # retrieve predicted percentage and label
  percent_pos = yhat[0,2]
  percent_neutral = yhat[0,1]
  percent_neg = yhat[0,0]
  max_percent = max(percent_pos, percent_neutral, percent_neg)
  if max_percent == percent_pos:
    return percent_pos, 'POSITIVE'
  elif max_percent == percent_neutral:
    return percent_neutral, 'NEUTRAL'
  else:
    return percent_neg, 'NEGATIVE'

In [ ]:
from keras.models import load_model


# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# load all revies
data = df.to_dict(orient='records')
data = [value for value in data if value['airline_sentiment_confidence'] >= 0.5]
train_docs, ytrain = load_clean_tweet(data, vocab)
# Convert ytrain to one-hot encoding
ytrain = to_categorical(ytrain, num_classes=3)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# load model
model = load_model('model.h5')
# evaluate model on training dataset
_, acc = model.evaluate(Xtrain, ytrain, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))


Vocabulary size: 5346
Maximum length: 22
Train Accuracy: 99.20


In [ ]:
# test negative
text = "@VirginAmerica didn't feel confortable for the service and the meals so smelly, not recommended"
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

# test neutral
text = "@VirginAmerica Are the hours of operation for the Club at SFO that are posted online current?"
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

# test positif
text = "@AmericanAir Thank you for the best service!"
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

Review: [@VirginAmerica didn't feel confortable for the service and the meals so smelly, not recommended]
Sentiment: NEGATIVE (92.315%)
Review: [@VirginAmerica Are the hours of operation for the Club at SFO that are posted online current?]
Sentiment: NEUTRAL (100.000%)
Review: [@AmericanAir Thank you for the best service!]
Sentiment: POSITIVE (91.683%)
